OLD NOTEBOOK: SEE osm_pbf_power_data_extractor.py which does everything.

In [78]:
import os, sys, time
#IMPORTANT: RUN SCRIPT FROM THIS SCRIPTS DIRECTORY i.e data_exploration/ TODO: make more robust
##os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append('../../scripts')
from iso_country_codes import AFRICA_CC



import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
import geoplot
import matplotlib.pyplot as plt
from iso_country_codes import AFRICA_CC
from osm_pbf_power_data_extractor import convert_pd_to_gdf_lines, convert_pd_to_gdf

import logging
logger = logging.getLogger(__name__)

# SUBSTATIONS

In [71]:
#----------- SUBSTATIONS -----------
# Load uncleaned data
df_all_substations = pd.read_csv(os.getcwd()+"/data/africa_all_substations.csv")
# Clean
df_all_substations = df_all_substations.reset_index(drop=True)
df_all_substations = df_all_substations.dropna(subset=['tags.voltage']) # Drop any substations with Voltage = N/A

#~ Generate Files

#CSV
# outputfile_partial = os.path.join(os.getcwd(),'data','africa_all'+'_substations'+'_cleaned.')
# df_all_substations.to_csv(outputfile_partial + 'csv') # Generate CSV

#GeoJSON
##gdf_substations = convert_pd_to_gdf(df_all_substations)
##gdf_substations.to_file(outputfile_partial+'geojson', driver="GeoJSON")  # Generate GeoJson


#display(df_all_substations)

# LINES 

### Check old unique values

### Import data and create final dataframe layout

In [143]:
# Load raw data
df_all_lines = gpd.read_file(os.getcwd()+"/data/africa_all_lines.geojson") 

# Modification - create final dataframe layout
df_all_lines = df_all_lines.rename(
    columns = {
        "id": "line_id",
        "tags.voltage": "voltage",
        "tags.circuits": "circuits",
        "tags.cables": "cables",
        "tags.frequency": "tag_frequency",
        "tags.power": "tag_type",
        "lonlat": "geometry",
    }
)

# Add NaN as default
df_all_lines["bus0"] = np.nan
df_all_lines["bus1"] = np.nan
df_all_lines["length"] = np.nan
df_all_lines["underground"] = np.nan
df_all_lines["under_construction"] = np.nan

#Rearrange columns
clist = ["line_id","bus0","bus1","voltage","circuits","length","underground",
         "under_construction","tag_type","tag_frequency","geometry"]
df_all_lines = df_all_lines[clist]

display(df_all_lines)

,line_id,bus0,bus1,voltage,circuits,length,underground,under_construction,tag_type,tag_frequency,geometry
0,610844602,NaN,NaN,None,None,NaN,NaN,NaN,line,None,"LINESTRING (4.06634 35.20694, 4.06813 35.20668..."
1,262871070,NaN,NaN,66000,None,NaN,NaN,NaN,line,None,"LINESTRING (6.46852 30.52800, 6.47092 30.52732..."
2,655094635,NaN,NaN,220000,None,NaN,NaN,NaN,line,50,"LINESTRING (-0.47397 35.72051, -0.47585 35.721..."
3,378839793,NaN,NaN,60000,None,NaN,NaN,NaN,line,50,"LINESTRING (-0.06154 35.28141, -0.06144 35.281..."
4,382460532,NaN,NaN,400000,None,NaN,NaN,NaN,line,None,"LINESTRING (2.07982 36.57460, 2.07938 36.57236..."
...,...,...,...,...,...,...,...,...,...,...,...
21420,651546006,NaN,NaN,330000,None,NaN,NaN,NaN,line,50,"LINESTRING (31.37627 -17.30964, 31.37637 -17.3..."
21421,651760976,NaN,NaN,132000,None,NaN,NaN,NaN,line,50,"LINESTRING (29.77979 -18.75640, 29.78005 -18.7..."
21422,200790530,NaN,NaN,330000,None,NaN,NaN,NaN,line,None,"LINESTRING (31.19731 -18.06076, 31.19709 -18.0..."
21423,651760977,NaN,NaN,132000,None,NaN,NaN,NaN,line,50,"LINESTRING (29.83323 -18.90983, 29.83308 -18.9..."


### Clean voltage

In [149]:
df = df_all_lines.dropna(subset=['voltage']) # Drop any lines with Voltage = N/A

#Split semicolon separated cells i.e. [66000;220000] and create rows
lst_col = 'voltage'
x = df.assign(**{lst_col:df[lst_col].str.split(';')})

x = pd.DataFrame({
    col:np.repeat(x[col].values, x[lst_col].str.len())
    for col in x.columns.difference([lst_col])
    }).assign(**{lst_col:np.concatenate(x[lst_col].values)})[x.columns.tolist()]





# df_all_lines['voltage'] = df_all_lines['voltage'].str.split('*').str[0] # handle special case in raw data 2*200000
# df_all_lines['voltage'] = df_all_lines['voltage'].apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()

display(df_all_lines)

,line_id,bus0,bus1,voltage,circuits,length,underground,under_construction,tag_type,tag_frequency,geometry
0,610844602,NaN,NaN,None,None,NaN,NaN,NaN,line,None,"LINESTRING (4.06634 35.20694, 4.06813 35.20668..."
1,262871070,NaN,NaN,66000,None,NaN,NaN,NaN,line,None,"LINESTRING (6.46852 30.52800, 6.47092 30.52732..."
2,655094635,NaN,NaN,220000,None,NaN,NaN,NaN,line,50,"LINESTRING (-0.47397 35.72051, -0.47585 35.721..."
3,378839793,NaN,NaN,60000,None,NaN,NaN,NaN,line,50,"LINESTRING (-0.06154 35.28141, -0.06144 35.281..."
4,382460532,NaN,NaN,400000,None,NaN,NaN,NaN,line,None,"LINESTRING (2.07982 36.57460, 2.07938 36.57236..."
...,...,...,...,...,...,...,...,...,...,...,...
21420,651546006,NaN,NaN,330000,None,NaN,NaN,NaN,line,50,"LINESTRING (31.37627 -17.30964, 31.37637 -17.3..."
21421,651760976,NaN,NaN,132000,None,NaN,NaN,NaN,line,50,"LINESTRING (29.77979 -18.75640, 29.78005 -18.7..."
21422,200790530,NaN,NaN,330000,None,NaN,NaN,NaN,line,None,"LINESTRING (31.19731 -18.06076, 31.19709 -18.0..."
21423,651760977,NaN,NaN,132000,None,NaN,NaN,NaN,line,50,"LINESTRING (29.83323 -18.90983, 29.83308 -18.9..."


In [148]:
x['voltage'].unique()



array(['66000', '220000', '60000', '400000', '3000', '22000', '90000',
       '30000', '150000', '225000', '132000', '2*220000', '350000',
       '110000', '330000', '161000', '70000', '120000', '500000', '0',
       '11000', '63000', '230000', '45000', '115000', '88000', 'medium',
       'high-medium', '30', '30_ك_ف', '30K_V', '30_KV', '30ك_ف', 'KV30',
       't', '30kv', '69000', '30_K_V', '25000', '138000', '33000',
       '20000', '400', '22500', '533000', '275000', '63KvA', '19000',
       '11', '33', '765000', '44000', '380000', '220', '15000', '150000.',
       'low', 'Avenida La Esperanza'], dtype=object)

In [139]:
df_all_lines['voltage'].explode().unique()

array([6.60e+04, 2.20e+05, 6.00e+04, 4.00e+05,      nan, 3.00e+03,
       2.20e+04, 9.00e+04, 3.00e+04, 1.50e+05, 2.25e+05, 1.32e+05,
       2.00e+00, 3.50e+05, 1.10e+05, 3.30e+05, 1.61e+05, 7.00e+04,
       1.20e+05, 5.00e+05, 0.00e+00, 1.10e+04, 6.30e+04, 2.30e+05,
       4.50e+04, 1.15e+05, 8.80e+04, 3.00e+01, 6.90e+04, 2.50e+04,
       1.38e+05, 3.30e+04, 2.00e+04, 4.00e+02, 2.25e+04, 5.33e+05,
       2.75e+05, 1.90e+04, 1.10e+01, 3.30e+01, 7.65e+05, 4.40e+04,
       3.80e+05, 2.20e+02, 1.50e+04])

In [133]:
if df_all_lines['voltage'] contains *:
    df_all_lines['voltage'].str.split('*').str[1]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
df_all_lines = df_all_lines.dropna(subset=['tags.voltage']) # Drop any lines with Voltage = N/A
df_all_lines = df_all_lines.rename(columns = {'tags.voltage':"voltage_V"}) 
df_all_lines['voltage_V'] = df_all_lines['voltage_V'].str.split('*').str[0] #just keeps the 
df_all_lines['voltage_V'] = df_all_lines['voltage_V']# .str.split(';').str[0]
df_all_lines['voltage_V'] = df_all_lines['voltage_V'].apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna() ## if cell can't converted to float -> drop
df_all_lines = df_all_lines[df_all_lines.voltage_V > 10000]

In [113]:
X = df_all_lines.voltage

print(X.unique())
print()
print("Datatype:", X.dtype)

['66000' '220000' '60000' '400000' '66000;66000' '3000' '22000' '90000'
 '30000' '220000;220000' '150000' '225000' '132000' '2*220000' '350000'
 '110000' '330000' '161000' '70000' '120000' '500000' '0' '11000' '63000'
 '230000' '220000;66000' '220000;110000' '45000' '115000' '220000;132000'
 '88000' 'medium' 'high-medium' '30' '30_ك_ف' '30K_V' '30_KV' '30ك_ف'
 'KV30' '30;t' '30kv' '69000' '30_K_V' '25000' '138000' '33000' '20000'
 '400' '22500' '533000' '275000' '63KvA' '19000' '11' '330000;132000' '33'
 '765000' '44000' '380000' '132000;66000' '400000;132000' '220' '15000'
 '150000.' 'low' 'Avenida La Esperanza']

Datatype: object


In [ ]:
## Generate Files
## CSV
# outputfile_partial = os.path.join(os.getcwd(), 'data', 'africa_all'+'_lines'+'_cleaned.')  
# df_all_lines.to_csv(outputfile_partial + 'csv')  # Generate CSV
## GeoJSON
# gdf_lines = convert_pd_to_gdf_lines(df_all_lines, simplified=True)
# gdf_lines.to_file(outputfile_partial+'geojson',
#             driver="GeoJSON")  # Generate GeoJson


# GENERATORS

In [72]:
# ----------- Generator -----------

#Load uncleaned data
df_all_generators = pd.read_csv(os.getcwd()+"/data/africa_all_generators.csv")

#Clean data
df_all_generators = df_all_generators.reset_index(drop=True)
df_all_generators = df_all_generators[df_all_generators['tags.generator:output:electricity'].astype(str).str.contains('MW')] #removes boolean 
df_all_generators['tags.generator:output:electricity'] = df_all_generators['tags.generator:output:electricity'].str.extract('(\d+)').astype(float)
df_all_generators = df_all_generators.rename(columns = {'tags.generator:output:electricity':"power_output_MW"})


## Generate Files

#CSV
#outputfile_partial = os.path.join(os.getcwd(),'data','africa_all'+'_generators'+'_cleaned.')
#df_all_generators.to_csv(outputfile_partial + 'csv') # Generate CSV

#GeoJSON
# gdf_generators = convert_pd_to_gdf(df_all_generators)
# gdf_generators.to_file(outputfile_partial+'geojson', driver="GeoJSON")  # Generate GeoJson


#display(df_all_generators)